Hamilton,W.L. *Graph Representation Learning*. 2020

Basic GNN은 여러 가지 방법으로 개선되고 일반화될 수 있다. 여기서는 aggregation과 update 작업을 일반화하고 개선할 수 있는 방법에 대해 알아보자.

# Generalized Neighborhood Aggregation

## Neighborhood Normalization

가장 기본적인 이웃 집계(aggreagation)는 단순히 이웃 임베딩의 합을 취하는 것이나, 이 방식은 불안정하고 노드 degree에 매우 민감할 수 있다. 예를 들어, 노드 $u$가 노드 $u'$보다 $100$배 많은 이웃(즉, 훨씬 더 높은 차수)이 있다고 가정 할 때 $\left\| \sum_{v \in \mathcal{N}(u)} h_v \right\| \gg \left\| \sum_{v' \in \mathcal{N}(u')} h_{v'} \right\|$라고 합리적으로 기대할 수 있다. 이러한 급격한 크기 차이로 인해 수치적 불안정성과 최적화의 어려움이 발생할 수 있다.

해결 방법으로 관련된 포함된 노드 degree에 따라 집계를 단순히 정규화하는 방법이 있다.

- 가장 단순한 방법은 합 대신 평균화 하는 것이다:

$$ \mathbf{m}_{\mathcal{N}(u)} = \frac{\sum_{v \in \mathcal{N}(u)} \mathbf{h}_v}{|\mathcal{N}(u)|} $$

- Kipf and Welling [2016a]에서 사용된 다음과 같은 *대칭 정규화*(*symmetric normalization*)와 같은 다른 정규화 factor도 있다:

$$ \mathbf{m}_{\mathcal{N}(u)} = \sum_{v \in \mathcal{N}(u)} \frac{\mathbf{h}_v}{\sqrt{|\mathcal{N}(u)| |\mathcal{N}(v)|}} $$

특히, 대칭 정규화 집계와 기본 GNN 업데이트 함수를 결합하면 spectral graph convolution의 1차(first-order) 근사(approximation)가 생성된다.

**Graph Convolutional Networks (GCNs)**

가장 널리 사용되는 기준이 되는 그래프 신경망 모델 중 하나인 그래프 컨볼루션 네트워크(**GCN**)는 대칭 정규화 집계와 self-loop 업데이트 접근 방식을 사용한다.

GCN은 메시지 전달 함수를 다음과 같이 정의한다:

$$\mathbf{h}^{(k)}_u = \sigma \left( \mathbf{W}^{(k)} \sum_{v \in \mathcal{N}(u) \cup \{u\}} \frac{\mathbf{h}_v}{\sqrt{|\mathcal{N}(u)| |\mathcal{N}(v)|}} \right)$$


정규화는 필요할까?

- GNN을 사용할 때 안정적이고 강력한 성능을 얻으려면 적절한 정규화가 필수적일 수 있으나, 정규화로 인해 정보가 손실될 수도 있다는 점에 유의해야 한다.
- 예를 들어, 정규화 후에는 학습된 임베딩을 사용하여 서로 다른 degree의 노드를 구별하는 것이 어렵거나 불가능할 수 있으며, 다양한 다른 구조 그래프 특징이 정규화에 의해 가려질 수 있다.
- 일반적으로 정규화는 노드 피쳐 정보가 구조적 정보보다 훨씬 더 유용하거나, 최적화 중에 불안정을 초래할 수 있는 노드 degree 범위가 매우 넓은 작업에서 유용하다.

## Set Aggregators

이웃 집계 연산은 기본적으로 집합 함수(set function)이다. 이웃 임베딩 집합 {$\mathbf{h}_v, \forall v \in \mathcal{N}(u)$}이 주어지고, 이를 싱글 벡터 $\mathbf{m}_{\mathcal{N}(u)}$에 매핑해야 한다. {$\mathbf{h}_v, \forall v \in \mathcal{N}(u)$}이 집합이라는 사실은 실제로 매우 중요하다: 노드 이웃에는 자연적인 순서가 없으며, 따라서 정의하는 모든 집계 함수는 *순열 불변*(*permutation invariant*)이어야 한다.

집계 함수를 정의하는 한 가지 원칙적인 접근 방식은 순열 불변 신경망 이론(permutaion invariant neural network)을 기반으로 한다. 

예를 들어, Zaheer et al. [2017]은 다음 형식의 집계 함수가 범용적 집합 함수 근사기(*universal set function approximator*)임을 보여준다:

$$\mathbf{m}_{\mathcal{N}(u)} = MLP_{\theta} \left( \sum_{v \in \mathcal{N}(u)} MLP_{\phi} (\mathbf{h}_v) \right)$$


## Message Passing with Self-Loops

neural message passing 접근 방식을 단순화하기 위해 입력 그래프에 self-loop를 추가하고 명시적 업데이트 단계를 생략하는 것이 일반적이다. 따라서 message passing을 단순히 다음과 같이 정의한다:

$$h_u^{(k)} = \text{AGGREGATE}\left(\left\{ h_v^{(k-1)}, \forall v \in \mathcal{N}(u) \cup \{u\} \right\}\right)$$


$\mathcal{N}(u) \cup \{u\}$ 집합 대상으로 aggregate하겠다는 것은 노드의 이웃뿐아니라 자기 자신도 포함한다는 것이다.

이 방법의 장점은 업데이트가 aggregation 방법을 통해 암시적으로 정의되기 때문에 더 이상 명시적 업데이트 함수를 정의할 필요가 없다는 것이다. 이러한 방식으로 전달되는 메시지를 단순화하면 종종 과적합을 완화할 수 있지만 노드의 이웃에서 오는 정보를 노드 자체의 정보와 구별할 수 없기 때문에 GNN의 표현성을 제한하기도 한다.

Basic GNN의 경우, self-loops를 추가하는 것은 $\mathbf{W}_{self}$ 행렬과 $\mathbf{W}_{neigh}$ 행렬 간에 매개변수를 공유하는 것과 동일하며, 다음과 같은 graph-level 업데이트를 제공한다.

$$\mathbf{H}^{(t)} = \sigma \left( (\mathbf{A} + \mathbf{I})\mathbf{H}^{(t-1)}\mathbf{W}^{(t)} \right)
$$


다음 포스트에서는 `AGGREGATE`, `UPDATE` 함수에 대해 더 알아보도록 하자.

# 참고자료

[1] Hamilton, W. L. (2020). *Graph Representation Learning.* Morgan & Claypool Publishers.